# Description

- Initialize with returns, predictions, target volatility, and oos start date
- Evaluate portfolios generated from the predictions

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import logging

import core.config as cconfig
import core.dataflow_model.model_evaluator as modeval
import core.dataflow_model.model_plotter as modplot
import core.dataflow_model.utils as cdmu
import helpers.dbg as dbg
import helpers.printing as hprint

In [2]:
dbg.init_logger(verbosity=logging.INFO)
#dbg.init_logger(verbosity=logging.DEBUG)

_LOG = logging.getLogger(__name__)

# _LOG.info("%s", env.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-304a7418-8455-4236-9080-0571b5f87e43.json'


In [3]:
# file_name = "/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_1/result_bundle.v2_0.pkl"

# import helpers.pickle_ as hpickle

# obj = hpickle.from_pickle(file_name)

In [4]:
#obj.keys()

In [5]:
# import helpers.introspection as hintro

# for k in obj.keys():
#     print(k, hintro.get_size_in_bytes(obj[k]))

In [6]:
#obj["payload"]["fit_result_bundle"]["result_df"]

# Notebook config

In [7]:
# exp_dir = "s3://eglp-spm-sasm/experiments/experiment.RH2Ef.v1_9-all.5T.20210831-004747.run1.tgz"
exp_dir = "/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test"
aws_profile = None

eval_config = cconfig.get_config_from_nested_dict(
    {
        "load_experiment_kwargs": {
            "src_dir": exp_dir,
            "file_name": "result_bundle.v2_0.pkl",
            "experiment_type": "ins_oos",
            "selected_idxs": None,
            "aws_profile": aws_profile,
        },
        "model_evaluator_kwargs": {
            "predictions_col": "mid_ret_0_vol_adj_clipped_2_hat",
            "target_col": "mid_ret_0_vol_adj_clipped_2",
            # "oos_start": "2017-01-01",
        },
        "bh_adj_threshold": 0.1,
        "resample_rule": "W",
        "mode": "ins",
        "target_volatility": 0.1,
    }
)

# Initialize ModelEvaluator and ModelPlotter

In [12]:
# # Load the data.
# #selected_idxs = list(range(4))
# result_bundles = cdmu.yield_experiment_artifacts(
#     eval_config["exp_dir"],
#     "result_bundle.pkl",
#     #selected_idxs=selected_idxs,
# )

load_config = eval_config["load_experiment_kwargs"].to_dict()

# Load only the columns needed by the ModelEvaluator.
load_config["load_rb_kwargs"] = {
    "columns": 
    [eval_config["model_evaluator_kwargs"]["target_col"],
    eval_config["model_evaluator_kwargs"]["predictions_col"]]
}
result_bundle_dict = cdmu.load_experiment_artifacts(**load_config)

# Build the ModelEvaluator.
evaluator = modeval.ModelEvaluator.from_result_bundle_dict(
    result_bundle_dict,
    #abort_on_error=False,
    abort_on_error=True,
    **eval_config["model_evaluator_kwargs"].to_dict(),
)
# Build the ModelPlotter.
plotter = modplot.ModelPlotter(evaluator)

# Load artifacts 'result_bundle.v2_0.pkl' from '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test'
Found 9 experiment subdirs in '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test'


Loading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Unpickling from '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_0/result_bundle.v2_0.pkl' ...
Unpickling from '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_0/result_bundle.v2_0.pkl' done (0.089 s)
Read '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_0/result_bundle.v2_0.pkl' (size=147.6 MB, time=0.1s)
Loaded '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_0/result_bundle.v2_0.pq' (size=40.1 MB, time=0.1s)
Unpickling from '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_1/result_bundle.v2_0.pkl' ...
Unpickling from '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_1/result_bundle.v2_0.pkl' done (0.086 s)
Read '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_1/result_bundle.v2_0.pkl' (size=147.6 MB, time=0.1s)
Loaded '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_1/result_bundle.v2_0.pq' (size=34.8 MB, time=0.0s)
Unpickling from '/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_2/result_bundle.v2_0.pkl

# Analysis

In [9]:
pnl_stats = evaluator.calculate_stats(
    mode=eval_config["mode"], target_volatility=eval_config["target_volatility"]
)
display(pnl_stats)

AssertionError: 
################################################################################
* Failed assertion *
cond=None
################################################################################


## Model selection

In [ ]:
plotter.plot_multiple_tests_adjustment(
    threshold=eval_config["bh_adj_threshold"], mode=eval_config["mode"]
)

In [ ]:
# TODO(gp): Move this chunk of code in a function.
col_mask = (
    pnl_stats.loc["signal_quality"].loc["sr.adj_pval"]
    < eval_config["bh_adj_threshold"]
)
selected = pnl_stats.loc[:, col_mask].columns.to_list()
not_selected = pnl_stats.loc[:, ~col_mask].columns.to_list()

print("num model selected=%s" % hprint.perc(len(selected), pnl_stats.shape[1]))
print("model selected=%s" % selected)
print("model not selected=%s" % not_selected)

# Use `selected = None` to show all the models.

In [ ]:
plotter.plot_multiple_pnls(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

## Return correlation

In [ ]:
plotter.plot_correlation_matrix(
    series="returns",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

In [ ]:
plotter.plot_effective_correlation_rank(
    series="returns",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

## Model correlation

In [ ]:
plotter.plot_correlation_matrix(
    series="pnl",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

In [ ]:
plotter.plot_effective_correlation_rank(
    series="pnl",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

## Aggregate model

In [ ]:
pnl_srs, pos_srs, aggregate_stats = evaluator.aggregate_models(
    keys=selected,
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)
display(aggregate_stats)

In [ ]:
plotter.plot_sharpe_ratio_panel(keys=selected, mode=eval_config["mode"])

In [ ]:
plotter.plot_rets_signal_analysis(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)

In [ ]:
plotter.plot_performance(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)

In [ ]:
plotter.plot_rets_and_vol(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)

In [ ]:
assert 0

In [ ]:
plotter.plot_positions(
    keys=selected,
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)

In [ ]:
# Plot the returns and prediction for one or more models.
model_key = selected[:1]
plotter.plot_returns_and_predictions(
    keys=model_key,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)